In [1]:
!ls data

BGL_preprocessed.less_templates.csv  README.md
BGL_preprocessed.standard.csv	     sequences_less_templates.dat
labels_less_templates.txt	     X_train_less_sequences.dat


In [2]:
outf = "data/sequences_less_templates.dat"
test_labels = "data/labels_less_templates.txt"

infile = "data/BGL_preprocessed.less_templates.csv" #"data/BGL_preprocessed.log_structured.csv"

bening_label = "-"

WINDOW_SIZE = 10

In [3]:
import pandas as pd

df = pd.read_csv(infile)
df

,LineId,Label,Timestamp,Date,Node,Time,NodeRepeat,Type,Component,Level,Content,EventId,EventTemplate,ParameterList
0,1,-,1117838570,2005.06.03,R02-M1-N0-C:J12-U11,2005-06-03-15.42.50.363779,R02-M1-N0-C:J12-U11,RAS,KERNEL,INFO,instruction cache parity error corrected,3aa50e45,instruction cache parity error corrected,[]
1,2,-,1117838570,2005.06.03,R02-M1-N0-C:J12-U11,2005-06-03-15.42.50.527847,R02-M1-N0-C:J12-U11,RAS,KERNEL,INFO,instruction cache parity error corrected,3aa50e45,instruction cache parity error corrected,[]
2,3,-,1117838570,2005.06.03,R02-M1-N0-C:J12-U11,2005-06-03-15.42.50.675872,R02-M1-N0-C:J12-U11,RAS,KERNEL,INFO,instruction cache parity error corrected,3aa50e45,instruction cache parity error corrected,[]
3,4,-,1117838570,2005.06.03,R02-M1-N0-C:J12-U11,2005-06-03-15.42.50.823719,R02-M1-N0-C:J12-U11,RAS,KERNEL,INFO,instruction cache parity error corrected,3aa50e45,instruction cache parity error corrected,[]
4,5,-,1117838570,2005.06.03,R02-M1-N0-C:J12-U11,2005-06-03-15.42.50.982731,R02-M1-N0-C:J12-U11,RAS,KERNEL,INFO,instruction cache parity error corrected,3aa50e45,instruction cache parity error corrected,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4747958,4747959,KERNSOCK,1136390405,2006.01.04,R00-M0-NC-I:J18-U11,2006-01-04-08.00.05.167045,R00-M0-NC-I:J18-U11,RAS,KERNEL,FATAL,idoproxy communication failure: socket closed,197e3bb9,<*> <*> <*> socket closed,"['idoproxy', 'communication failure:']"
4747959,4747960,KERNSOCK,1136390405,2006.01.04,R36-M0-NC-I:J18-U11,2006-01-04-08.00.05.185942,R36-M0-NC-I:J18-U11,RAS,KERNEL,FATAL,idoproxy communication failure: socket closed,197e3bb9,<*> <*> <*> socket closed,"['idoproxy', 'communication failure:']"
4747960,4747961,KERNSOCK,1136390405,2006.01.04,R30-M0-NC-I:J18-U11,2006-01-04-08.00.05.186013,R30-M0-NC-I:J18-U11,RAS,KERNEL,FATAL,idoproxy communication failure: socket closed,197e3bb9,<*> <*> <*> socket closed,"['idoproxy', 'communication failure:']"
4747961,4747962,KERNSOCK,1136390405,2006.01.04,R31-M0-NC-I:J18-U11,2006-01-04-08.00.05.204230,R31-M0-NC-I:J18-U11,RAS,KERNEL,FATAL,idoproxy communication failure: socket closed,197e3bb9,<*> <*> <*> socket closed,"['idoproxy', 'communication failure:']"


In [4]:
def map_label(x: str):
    if x=="-":
        return 0
    return 1

def label_sequence(x: list):
    #if 1 in set(x):
    #    return 1
    if x[-1] == 1:
        return 1
    elif 1 in set(x):
        return -1
    return 0

In [5]:
benign_seq = list()
malign_seq = list()

alphabet = dict()

In [6]:
import copy

for gname, group in df.groupby("Node"):
    benign_seq_tmp = list()
    malign_seq_tmp = list()

    event_window = list()
    label_window = list()
    for i, (event_id, label) in enumerate(zip(group.EventId, group.Label)):
        label = map_label(label)
        if not event_id in alphabet:
            alphabet[event_id] = len(alphabet)
        
        event_window.append(alphabet[event_id])
        label_window.append(copy.copy(label))
        if len(event_window) == WINDOW_SIZE:
            seq_label = label_sequence(label_window)
            if seq_label == -1:
                del event_window[0]
                del label_window[0]
                continue
            elif seq_label == 1:
                malign_seq_tmp.append(copy.copy(event_window))
            else:
                benign_seq_tmp.append(copy.copy(event_window))
            del event_window[0]
            del label_window[0]
    benign_seq.extend(copy.copy(benign_seq_tmp))
    malign_seq.extend(copy.copy(malign_seq_tmp))
    
len(benign_seq), len(malign_seq), len(alphabet)

(3296098, 754306, 1759)

# train-test-split

In [7]:
import numpy as np
import random

y_test = np.ones((len(malign_seq), 1))
y_test = np.vstack((y_test, np.zeros((len(benign_seq), 1)) ))
y_test = list(y_test)

malign_seq.extend(benign_seq)

len(y_test), len(malign_seq)

(4050404, 4050404)

In [8]:
with open(outf, "wt") as test_fh, open(test_labels, "wt") as label_fh:
    test_fh.write("{} {}\n".format(len(malign_seq), len(alphabet)))
    for line, label in zip(malign_seq, y_test):
        test_fh.write("{} {} {}\n".format(0, len(line), " ".join([str(x) for x in line])))
        label_fh.write("{}\n".format(int(label[0])))